In [1]:
import os
from typing import Tuple, Callable, Dict, Optional, List

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from Recommenders.DataIO import DataIO
from skopt.space import Real, Integer, Categorical
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender,MultiThreadSLIM_SLIM_S_ElasticNetRecommender



columns_name = ['user_id','item_id','impression_list','data']

dataset_l = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_ICM_length.csv', sep=',')
dataset_t = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_ICM_type.csv', sep=',')
dataset = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/interactions_and_impressions.csv', sep=',')
test_users = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_target_users_test.csv',sep = ',')
dataset.columns = columns_name

c:\Users\loren\anaconda3\envs\RecSysFramework\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
len(dataset.item_id.unique()),dataset.item_id.unique().min(),dataset.item_id.unique().max()

(24507, 0, 24506)

In [3]:
from Data_manager.Build_Matrix.Data_import import build_URM,build_URM_ICM,build_URM_impression,build_URM_ICM_onlyURM_item
URM,ICM=build_URM_ICM_onlyURM_item(dataset, dataset_t, dataset_l)
URM_1= build_URM(dataset)

In [4]:
URM

<41629x24507 sparse matrix of type '<class 'numpy.int64'>'
	with 1554640 stored elements in Compressed Sparse Row format>

In [5]:
URM_1

<41629x24507 sparse matrix of type '<class 'numpy.int64'>'
	with 1554640 stored elements in Compressed Sparse Row format>

In [6]:
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 816 ( 2.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 310 ( 0.7%) Users that have less than 1 test interactions


In [7]:
import os

output_folder_path = "result_experiments_MultiThreadSLIM_SLIM_S_ElasticNetRecommender"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 100
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

hyperparameters_range_dictionary = {
                "topK": Integer(5, 1000),
                "l1_ratio": Real(low = 1e-5, high = 1.0, prior = 'log-uniform'),
                "alpha": Real(low = 1e-3, high = 1.0, prior = 'uniform'),
            }

earlystopping_keywargs = {"validation_every_n": 5,
                              "stop_on_validation": True,
                              "evaluator_object": URM_validation,
                              "lower_validations_allowed": 5,
                              "validation_metric": metric_to_optimize,
                              }
recommender_input_args = SearchInputRecommenderArgs(
                CONSTRUCTOR_POSITIONAL_ARGS = [URM_train,ICM],
                CONSTRUCTOR_KEYWORD_ARGS = {},
                FIT_POSITIONAL_ARGS = [],
                FIT_KEYWORD_ARGS = {},
                EARLYSTOPPING_KEYWORD_ARGS = {},
            )

recommender_class = MultiThreadSLIM_SLIM_S_ElasticNetRecommender


hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                        evaluator_validation=evaluator_validation,
                                        evaluator_test=evaluator_test)

In [8]:
hyperparameterSearch.search(recommender_input_args,
                    hyperparameter_search_space = hyperparameters_range_dictionary,
                    n_cases = n_cases,
                    n_random_starts = n_random_starts,
                    save_model = "last",
                    output_folder_path = output_folder_path, # Where to save the results
                    output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                    metric_to_optimize = metric_to_optimize,
                    cutoff_to_optimize = cutoff_to_optimize,
                    resume_from_saved = True,
                    recommender_input_args_last_test=recommender_input_args
                    )

SearchBayesianSkopt: Resuming 'SLIMElasticNetRecommender'... Loaded 33 configurations.
Iteration No: 1 started. Evaluating function at random point.


  0%|          | 0/24507 [00:00<?, ?it/s]

Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.6080
Function value obtained: -0.0128
Current minimum: -0.0128
Iteration No: 2 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 5, 'l1_ratio': 5.59101442900514e-05, 'alpha': 0.001}


100%|█████████▉| 24504/24507 [1:14:24<00:00,  5.49it/s] 


EvaluatorHoldout: Processed 40813 (100.0%) in 27.20 sec. Users per second: 1501
SearchBayesianSkopt: Config 33 is suboptimal. Config: {'topK': 5, 'l1_ratio': 5.59101442900514e-05, 'alpha': 0.001} - results: PRECISION: 0.0212457, PRECISION_RECALL_MIN_DEN: 0.0347041, RECALL: 0.0329112, MAP: 0.0060401, MAP_MIN_DEN: 0.0101237, MRR: 0.0477034, NDCG: 0.0266122, F1: 0.0258221, HIT_RATE: 0.1749688, ARHR_ALL_HITS: 0.0536290, NOVELTY: 0.0054854, AVERAGE_POPULARITY: 0.0582629, DIVERSITY_MEAN_INTER_LIST: 0.9937285, DIVERSITY_HERFINDAHL: 0.9993704, COVERAGE_ITEM: 0.6568735, COVERAGE_ITEM_HIT: 0.0408455, ITEMS_IN_GT: 0.9946546, COVERAGE_USER: 0.9803983, COVERAGE_USER_HIT: 0.1715391, USERS_IN_GT: 0.9803983, DIVERSITY_GINI: 0.1730343, SHANNON_ENTROPY: 12.1908772, RATIO_DIVERSITY_HERFINDAHL: 0.9996782, RATIO_DIVERSITY_GINI: 0.3310917, RATIO_SHANNON_ENTROPY: 0.8993785, RATIO_AVERAGE_POPULARITY: 1.1420212, RATIO_NOVELTY: 0.3992514, 



...

EvaluatorHoldout: Processed 40813 (100.0%) in 26.47 sec. Users per second: 1542
SearchBayesianSkopt: Config 42 is suboptimal. Config: {'topK': 902, 'l1_ratio': 0.9361694666316012, 'alpha': 0.9864718238345402} - results: PRECISION: 0.0001372, PRECISION_RECALL_MIN_DEN: 0.0002203, RECALL: 0.0001878, MAP: 0.0000358, MAP_MIN_DEN: 0.0000517, MRR: 0.0003575, NDCG: 0.0001635, F1: 0.0001586, HIT_RATE: 0.0013721, ARHR_ALL_HITS: 0.0003575, NOVELTY: 0.0063053, AVERAGE_POPULARITY: 0.0041086, DIVERSITY_MEAN_INTER_LIST: 0.3602329, DIVERSITY_HERFINDAHL: 0.9360224, COVERAGE_ITEM: 0.0038356, COVERAGE_ITEM_HIT: 0.0008569, ITEMS_IN_GT: 0.9946546, COVERAGE_USER: 0.9803983, COVERAGE_USER_HIT: 0.0013452, USERS_IN_GT: 0.9803983, DIVERSITY_GINI: 0.0006570, SHANNON_ENTROPY: 4.3075433, RATIO_DIVERSITY_HERFINDAHL: 0.9363107, RATIO_DIVERSITY_GINI: 0.0012571, RATIO_SHANNON_ENTROPY: 0.3177878, RATIO_AVERAGE_POPULARITY: 0.0805337, RATIO_NOVELTY: 0.4589287, 



  0%|          | 0/24507 [00:00<?, ?it/s]

Iteration No: 11 ended. Evaluation done at random point.
Time taken: 450.9303
Function value obtained: -0.0000
Current minimum: -0.0128
Iteration No: 12 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 5, 'l1_ratio': 1e-05, 'alpha': 0.7224501864601771}


 27%|██▋       | 6728/24507 [10:03<20:34, 14.41it/s]

KeyboardInterrupt: 